# KRAS: Effects of Mutation

<b> Standard imports for playing with and plotting data frames. </b>

In [9]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns

<b> Import CPTAC </b>

In [10]:
import CPTAC

Loading Clinical Data...
Loading Proteomics Data...
Loading Transcriptomics Data...
Loading CNA Data...
Loading Phosphoproteomics Data...
Loading Somatic Data...

 ******PLEASE READ******


In [11]:
somatic_mutations = CPTAC.get_somatic()
proteomics = CPTAC.get_proteomics()
phos = CPTAC.get_phosphoproteomics()

<b> List of proteins that interact with KRAS (according to STRING) </b>

In [12]:
protList = ['EGRF', 'PIK3CA', 'ERBB2', 'ARAF', 'BRAF', 'MAPK1', 'MAPK3', 'MAP2K1', 'SOS1', 'RAF1']

### Proteome abundance of proteins that interact with KRAS

In [13]:
gene = 'KRAS'
sigList = [];

pcutoff = 0.05/len(protList)
print(pcutoff)

genedf = somatic_mutations[gene].to_frame()
for protein in protList:
    if protein in proteomics.columns:
        proteindf = proteomics[protein].to_frame()
        proteindfName = protein + " protein"
        proteindf.columns = [proteindfName]
        cross = genedf.add(proteindf, fill_value=0).dropna(axis=0)
        mutated = cross.loc[cross[gene] == 1.0]
        wt = cross.loc[cross[gene] == 0.0]
        ttest = scipy.stats.ttest_ind(mutated[proteindfName], wt[proteindfName])
        if ttest[1] <= pcutoff:
            sigList.append(protein)
            print("Test for " + protein + ": ")
            print(ttest)

0.005


AttributeError: module 'CPTAC' has no attribute 'compare_mutations'

<b> Print out the list of proteins  </b>

In [6]:
print(sigList)

[]


### Phosphoproteome abundance of interacting proteins

In [8]:
phosProtList = ['EGRF', 'PIK3CA', 'ERBB', 'ARAF', 'BRAF', 'MAPK', 'MAPK', 'MAP2K', 'SOS', 'RAF']

In [9]:
gene = 'KRAS'
genedf = somatic_mutations[gene].to_frame()
sigPhosResults = [];

for protein in phosProtList:
    sites = phos.filter(regex=protein)
    genedf = genedf.add(sites, fill_value=0)

mutated = genedf.loc[genedf[gene] == 1.0]
wt = genedf.loc[genedf[gene] == 0.0]

pcutoff = 0.05 / len(genedf.columns)
print(pcutoff)

for loc in genedf.columns:
    if not loc == gene:
        mutsitedf = mutated[[gene, loc]].dropna()
        wtsitedf = wt[[gene, loc]].dropna()
        ttest = scipy.stats.ttest_ind(mutsitedf[loc], wtsitedf[loc])
        if(ttest[1] <= pcutoff):
            sigPhosResults.append(loc)
            print('Results for ' + loc + ': ')
            print(ttest)
 

0.0006172839506172839
Results for ARAF-S260: 
Ttest_indResult(statistic=5.1539316473499825, pvalue=1.665516443552024e-06)
Results for ARAF-T256: 
Ttest_indResult(statistic=3.638629367252144, pvalue=0.0005162594349961674)
Results for SOS1-S1229: 
Ttest_indResult(statistic=3.937102285266522, pvalue=0.00018474397689668083)
Results for TRAF2-S102: 
Ttest_indResult(statistic=3.5556715706580255, pvalue=0.0005978116991508809)
Results for TRAFD1-S480: 
Ttest_indResult(statistic=3.9254242658334673, pvalue=0.0002471818737979513)
